In [1]:
!ls

Choose B-cell epitopes.ipynb
Extract SARS2 literature B-cell epitopes.ipynb
Visualize B-cell epitopes.ipynb
accessible-epitopes-vs-mutations-and-glycosites.png
accessible-linear-bcell-epitopes-grouped-by-sequence.csv
accessible-linear-bcell-epitopes-grouped-merged-filtered.csv
accessible-linear-bcell-epitopes-merged-overlap.csv
full-spike-glycosylated-accessibility-mutations-glycosites.png
full-spike-glycosylated-number-of-sources-mutations-glycosites.png
linear-bcell-epitopes-SARS2-S-with-filters.csv
linear-bcell-epitopes-SARS2-S.csv
linear-bcell-epitopes-SARS2.csv
linear-bcell-epitopes.csv
literature-linear-bcell-epitopes-04-28-2020.csv
literature-linear-bcell-epitopes-05-18-2020.csv
spike-bcell-epitopes-colored-by-number-of-sources.png


In [2]:
df = pd.read_csv("literature-linear-bcell-epitopes-05-18-2020.csv"); 
df


,Source,Virus,Protein,Start,End,Animal,Isotype,Immunization,Monoclonal Name,In vitro function,Vaccine function,Notes,Sequence
0,Berry 2010,SARS,S,460,476,Human,IgG,Infection,F26G18,Neutralizing,NaN,54% identity with \nPCNGVEGFNCYFPLQSYGFQPTNGVG...,NaN
1,Berry 2010,SARS,S,612,620,Human,IgG,Infection,F26G8,Non-neutralizing,NaN,NaN,NaN
2,Hu 2005,SARS,S,471,503,Human,IgG,Infection,Polyclonal,Neutralizing,NaN,"binds ACE2, in RDB",NaN
3,Hu 2005,SARS,S,604,625,Human,IgG,Infection,Polyclonal,Non-neutralizing,NaN,"""high variability of S471-503 and \nS604-625 i...",NaN
4,Hu 2005,SARS,S,1164,1191,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,ReScan,SARS-CoV-2,S,552,589,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,NaN
137,ReScan,SARS-CoV-2,N,362,399,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,NaN
138,ReScan,SARS-CoV-2,N,210,247,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,NaN
139,ReScan,SARS-CoV-2,N,153,190,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,NaN


In [3]:
df["Length"] = df["End"] - df["Start"] + 1;
df["Length"].value_counts()


15     55
5      19
38      8
17      7
23      6
22      4
25      4
300     3
33      3
31      2
7       2
18      2
19      2
20      2
28      2
226     2
29      2
54      1
108     1
9       1
10      1
90      1
63      1
57      1
459     1
32      1
21      1
42      1
39      1
35      1
26      1
27      1
48      1
Name: Length, dtype: int64

In [4]:
def parse_fasta(filename):
    result = {}
    with open(filename) as f:
        current_id = None
        lines = []
        for l in f:
            l = l.strip()
            if not l:
                continue
            elif l.startswith(">"):
                if current_id:
                    result[current_id] = "".join(lines)
                current_id = l[1:].split("|")[0]
                lines = []
            else:
                lines.append(l)
        if current_id:
            result[current_id] = "".join(lines)
        return result

proteome = parse_fasta("../Figures/COVID/AA_sequence.txt")
S = proteome["S"]


In [5]:
# update sequences for any SARS2 epitopes which are missing sequences or have incorrect lengths
updates_epitope_sequences = []
n_updated = 0
for _, row in df.iterrows():
    s = row.Sequence
    if type(s) is not str:
        s = ""
    if row.Virus == "SARS-CoV-2":
        if len(s) != row.Length:
            s = proteome[row.Protein][row.Start - 1:row.End]
            assert len(s) == row.Length
            n_updated += 1
    updates_epitope_sequences.append(s)
df["Sequence"] = updates_epitope_sequences
print("Added %d sequences" % n_updated)

    

Added 10 sequences


In [6]:
df["Sequence_length"] = df.Sequence.str.len()

In [7]:
df[df.Sequence_length != df.Length]

,Source,Virus,Protein,Start,End,Animal,Isotype,Immunization,Monoclonal Name,In vitro function,Vaccine function,Notes,Sequence,Length,Sequence_length
0,Berry 2010,SARS,S,460,476,Human,IgG,Infection,F26G18,Neutralizing,NaN,54% identity with \nPCNGVEGFNCYFPLQSYGFQPTNGVG...,,17,0
1,Berry 2010,SARS,S,612,620,Human,IgG,Infection,F26G8,Non-neutralizing,NaN,NaN,,9,0
2,Hu 2005,SARS,S,471,503,Human,IgG,Infection,Polyclonal,Neutralizing,NaN,"binds ACE2, in RDB",,33,0
3,Hu 2005,SARS,S,604,625,Human,IgG,Infection,Polyclonal,Non-neutralizing,NaN,"""high variability of S471-503 and \nS604-625 i...",,22,0
4,Hu 2005,SARS,S,1164,1191,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,,28,0
5,Hu 2005,SARS,N,67,76,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,,10,0
6,Hu 2005,SARS,N,367,389,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,,23,0
7,He 2004,SARS,S,9,71,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,,63,0
8,He 2004,SARS,S,171,224,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,,54,0
9,He 2004,SARS,S,271,318,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,,48,0


In [8]:
df.to_csv("linear-bcell-epitopes.csv", index=False)

In [9]:
df_sars2 = df[df.Virus == "SARS-CoV-2"];
df_sars2.to_csv("linear-bcell-epitopes-SARS2.csv", index=False)

In [10]:
df_sars2_S = df_sars2[df_sars2.Protein == "S"];
df_sars2_S.to_csv("linear-bcell-epitopes-SARS2-S.csv", index=False)

In [11]:
df_sars2_S

,Source,Virus,Protein,Start,End,Animal,Isotype,Immunization,Monoclonal Name,In vitro function,Vaccine function,Notes,Sequence,Length,Sequence_length
35,Poh 2020,SARS-CoV-2,S,553,570,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,TESNKKFLPFQQFGRDIA,18,18
36,Poh 2020,SARS-CoV-2,S,809,826,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,PSKPSKRSFIEDLLFNKV,18,18
42,Wang 2020,SARS-CoV-2,S,26,30,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,PAYTN,5,5
43,Wang 2020,SARS-CoV-2,S,186,190,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,FKNLR,5,5
44,Wang 2020,SARS-CoV-2,S,356,360,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,KRISN,5,5
45,Wang 2020,SARS-CoV-2,S,456,460,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,FRKSN,5,5
46,Wang 2020,SARS-CoV-2,S,806,820,Human,"IgG,IgM",Infection,Polyclonal,NaN,NaN,IgM epitpe is 816-SFIED-820,LPDPSKPSKRSFIED,15,15
47,Wang 2020,SARS-CoV-2,S,1196,1200,Human,IgG,Infection,Polyclonal,NaN,NaN,NaN,SLIDL,5,5
48,Wang 2020,SARS-CoV-2,S,886,890,Human,IgM,Infection,Polyclonal,NaN,NaN,NaN,WTFGA,5,5
49,Wang 2020,SARS-CoV-2,S,1046,1050,Human,IgM,Infection,Polyclonal,NaN,NaN,NaN,GYHLM,5,5
